# Introduction

The notebook is intended to perform a Sales Forecast over the data of the [Tabular Playground Series - September 2022](https://www.kaggle.com/competitions/tabular-playground-series-sep-2022)

In [ ]:
# Import Standard Libraries
import pandas as pd

# Read Data

In [ ]:
# Read train and test data
train_data = pd.read_csv('./../../data/2022_09/train.csv')
test_data = pd.read_csv('./../../data/2022_09/test.csv')

In [ ]:
train_data.info()

In [ ]:
test_data.info()